# PD4 - Jan Smoleń

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import dalex as dx
import pickle
np.random.seed = 46
import shap
import xgboost as xgb
from sklearn.metrics import accuracy_score
import warnings
import plotly
warnings.filterwarnings('ignore')
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

## Wczytywanie danych

In [ ]:
aps=dx.datasets.load_apartments()
from sklearn.preprocessing import OneHotEncoder
aps=dx.datasets.load_apartments()
Xa=aps.drop("district", axis=1)
ya=aps["district"]
aps.head()

Ponieważ mamy użyć SVM, to potraktujemy to zadanie jako klasyfikację ze względu na dzielnicę.

In [ ]:
wines=pd.read_csv("winequality-red.csv")
wines["is_good"] = wines.apply(lambda row: 1 if row.quality > 5 else 0, axis = 1)
Xw = wines.drop(["quality", "is_good"], axis = 1)
yw = wines[["is_good"]]
wines.head()

Ramkę danych o winie potraktujemy jako zadanie klasyfikacji - czy wino jest dobre (ma ocenę powyżej 5) czy nie.

## Podział na zbiory testowe i treningowe

In [ ]:
from sklearn.model_selection import train_test_split
Xw_train, Xw_test, yw_train, yw_test = train_test_split(Xw, yw, test_size = 0.2, random_state = 1613)

In [ ]:
Xa_train, Xa_test, ya_train, ya_test = train_test_split(Xa, ya, test_size = 0.2, random_state = 1613)

## SVM

In [ ]:
from sklearn.svm import SVC
svm_a=SVC()
svm_a.fit(Xa_train, ya_train)
svm_w=SVC()
svm_w.fit(Xw_train, yw_train)

### Bazowe wyniki, bez standaryzacji

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [ ]:
ya_preds=svm_a.predict(Xa_test)
accuracy_score(ya_test, ya_preds)

In [ ]:
yw_preds=svm_w.predict(Xw_test)
accuracy_score(yw_test, yw_preds)

Bazowe SVM na surowych zbiorach osiąga bardzo słabe wyniki. 

### Po standaryzacji

In [ ]:
Xa=(Xa-Xa.mean())/Xa.std()
Xw=(Xw-Xw.mean())/Xw.std()
Xw_train, Xw_test, yw_train, yw_test = train_test_split(Xw, yw, test_size = 0.2, random_state = 1613)
Xa_train, Xa_test, ya_train, ya_test = train_test_split(Xa, ya, test_size = 0.2, random_state = 1613)
svm_a.fit(Xa_train, ya_train)
svm_w.fit(Xw_train, yw_train)


In [ ]:
ya_preds=svm_a.predict(Xa_test)
accuracy_score(ya_test, ya_preds)

In [ ]:
yw_preds=svm_w.predict(Xw_test)
accuracy_score(yw_test, yw_preds)

Samo standaryzowanie bardzo polepszyło wyniki naszych modeli - o ponad 10%.

## Trening

In [ ]:
svm_a_tuned=SVC(random_state=42)
c=[]  # wartości parametru C
gamma=[]  #wartości parametru gamma 
for i in range(-4, 5):      # orientacyjne wartości na podstawie informacji znalezionych w internecie
    c.append(10**i)
for i in range(-4, 5):
    gamma.append(10**i)
gamma.append("auto")
gamma.append("scale")
params = [{'C': c,   
        'gamma': gamma,
        'kernel': ["rbf", "linear"]}]
from sklearn.model_selection import RandomizedSearchCV
rs_svm_a=RandomizedSearchCV(svm_a_tuned, param_distributions=params, scoring='accuracy', cv=4, n_jobs=2)
#gs_svm=GridSearchCV(svm_a_tuned, param_grid=params, scoring='accuracy', cv=4, n_jobs=2)
rs_svm_a.fit(Xa_train, ya_train)
rs_svm_a.best_params_

In [ ]:
rs_svm_a_acc=accuracy_score(gs_svm.predict(Xa_test),ya_test)
rs_svm_a_acc

In [ ]:
svm_w_tuned=SVC(random_state=42)
c=[]  # wartości parametru C
gamma=[]  #wartości parametru gamma 
for i in range(-4, 5):      # orientacyjne wartości na podstawie informacji znalezionych w internecie
    c.append(10**i)
for i in range(-4, 5):
    gamma.append(10**i)
gamma.append("auto")
gamma.append("scale")
params = [{'C': c,   
        'gamma': gamma,
        'kernel': ["rbf", "linear"]}]
rs_svm_w=RandomizedSearchCV(svm_w_tuned, param_distributions=params, scoring='accuracy', cv=4, n_jobs=2)
rs_svm_w.fit(Xw_train, yw_train)
rs_svm_w.best_params_

In [ ]:
rs_svm_w_acc=accuracy_score(gs_w_svm.predict(Xw_test),yw_test)
rs_svm_w_acc

Po dosyć długim czasie trenowania, dostaliśmy wyniki takie same albo nawet nieznacznie gorsze od bazowego SVM na wystandaryzowanych danych.